In [1]:
import xml.sax
import subprocess
import mwparserfromhell
import re

In [2]:
# Function where ContentHandler looks for opening and closing tags title and text 
# and adds characters enclosed within them to the buffer
# content saved to a dict with tag as key

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):         #do we need timestamp?
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [3]:
data_path = r"/home/mmartinelli/project/corpora/wikidumps/itwiki-20210720-pages-articles-multistream.xml.bz2"
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

lst = []
# Iterating through compressed file
for i, line in enumerate(subprocess.Popen(['bzcat'], stdin = open(data_path), stdout = subprocess.PIPE).stdout):
    
    parser.feed(line)
    # Stop after nth article reached
    if len(handler._pages) > 20000:
        break

In [4]:
# Append all articles that have the strings defined in categories in wikified_dishes list 
# the list has tuples with [0] being the title and [1] being the text

wikified_dishes = []
categories = ["Categoria:Antipasti", "Categoria:Contorni", "Categoria:Dolci", "Categoria:Involtini", 
              "Categoria:Piatti unici", "Categoria:Primi piatti", "Categoria:Secondi piatti"]
for x in handler._pages:
    if any(cat in x[1] for cat in categories):
        wikified_dishes.append(x)
    else:
        pass
print(len(wikified_dishes))
#print(wikified_dishes)
        

3
[('Samsa', '{{Gastronomia \n |nome =Samsa  \n |immagine = Samsa in Karakol.jpg \n |didascalia = I triangoli prima della cottura \n |IPA =  \n |altri nomi =  \n |paese = Algeria \n |paese2 = Tunisia \n |paese3 = \n |regione =  \n |regione2 =  \n |creatore =  \n |diffusione =  \n |zona =  \n |categoria = dolce \n |riconoscimento =  \n |consorzio =  \n |ingredienti = mandorle e semi di sesamo  \n |varianti =  \n |calorie =  \n |cat =  \n }} \n La \'\'\'\'\'samsa\'\'\'\'\' è un dolce tipico della cucina algerina e tunisina costituito da alcuni strati di pasta simili alla sfoglia (\'\'malsouka\'\') e farcito con [[mandorla|mandorle]] tritate e semi di [[sesamo]] < ref name= " Labourdette " > {{cita libro|autore=Collectif, Petit Futé, , Dominique Auzias, Jean-Paul Labourdette|titolo=Algérie 2015 Petit Futé (avec cartes, photos + avis des lecteurs) |editore=Petit Futé|ISBN=2-7469-7837-7|anno=2015|pagina=122|url=https://books.google.it/books?id=SS-VBgAAQBAJ & pg=PA122|lingua=en}} < /ref > , 

In [5]:
# Create title list and append only titles (element 0 of tuples in wikified_dishes)

title_lst = [el[0] for el in wikified_dishes]
#print(title_lst)

# Create text list and append only texts (element 1 of tuples in wikified_dishes)

text_lst = [el[1] for el in wikified_dishes]
#print(text_lst)

['Samsa', 'Kebab', 'Minestrone alla genovese']
['{{Gastronomia \n |nome =Samsa  \n |immagine = Samsa in Karakol.jpg \n |didascalia = I triangoli prima della cottura \n |IPA =  \n |altri nomi =  \n |paese = Algeria \n |paese2 = Tunisia \n |paese3 = \n |regione =  \n |regione2 =  \n |creatore =  \n |diffusione =  \n |zona =  \n |categoria = dolce \n |riconoscimento =  \n |consorzio =  \n |ingredienti = mandorle e semi di sesamo  \n |varianti =  \n |calorie =  \n |cat =  \n }} \n La \'\'\'\'\'samsa\'\'\'\'\' è un dolce tipico della cucina algerina e tunisina costituito da alcuni strati di pasta simili alla sfoglia (\'\'malsouka\'\') e farcito con [[mandorla|mandorle]] tritate e semi di [[sesamo]] < ref name= " Labourdette " > {{cita libro|autore=Collectif, Petit Futé, , Dominique Auzias, Jean-Paul Labourdette|titolo=Algérie 2015 Petit Futé (avec cartes, photos + avis des lecteurs) |editore=Petit Futé|ISBN=2-7469-7837-7|anno=2015|pagina=122|url=https://books.google.it/books?id=SS-VBgAAQBAJ

In [6]:
# Parse text list

text_lst = [mwparserfromhell.parse(text) for text in text_lst]
#print(text_lst)


['{{Gastronomia \n |nome =Samsa  \n |immagine = Samsa in Karakol.jpg \n |didascalia = I triangoli prima della cottura \n |IPA =  \n |altri nomi =  \n |paese = Algeria \n |paese2 = Tunisia \n |paese3 = \n |regione =  \n |regione2 =  \n |creatore =  \n |diffusione =  \n |zona =  \n |categoria = dolce \n |riconoscimento =  \n |consorzio =  \n |ingredienti = mandorle e semi di sesamo  \n |varianti =  \n |calorie =  \n |cat =  \n }} \n La \'\'\'\'\'samsa\'\'\'\'\' è un dolce tipico della cucina algerina e tunisina costituito da alcuni strati di pasta simili alla sfoglia (\'\'malsouka\'\') e farcito con [[mandorla|mandorle]] tritate e semi di [[sesamo]] < ref name= " Labourdette " > {{cita libro|autore=Collectif, Petit Futé, , Dominique Auzias, Jean-Paul Labourdette|titolo=Algérie 2015 Petit Futé (avec cartes, photos + avis des lecteurs) |editore=Petit Futé|ISBN=2-7469-7837-7|anno=2015|pagina=122|url=https://books.google.it/books?id=SS-VBgAAQBAJ & pg=PA122|lingua=en}} < /ref > , poi fritto o

In [7]:
# Clean texts (although no apparent change from mwparserfromhell.parse())

text_lst = [text.strip_code().strip() for text in text_lst]
#print(text_lst)


['La samsa è un dolce tipico della cucina algerina e tunisina costituito da alcuni strati di pasta simili alla sfoglia (malsouka) e farcito con mandorle tritate e semi di sesamo < ref name= " Labourdette " >  < /ref > , poi fritto o cotto al forno. Si prepara uno sciroppo di zucchero nel quale si mette la samsa per zuccherarla. La samsa oppure lo sciroppo devono essere ancora caldi per permettere un buon assorbimento < ref name= " Sherazade " >  < /ref > < ref name= " Amourdecuisine " >  < /ref > . \n \n == Note == \n < references / > \n \n == Voci correlate == \n * Samosa \n \n == Altri progetti == \n  \n \n  \n \n Categoria:Cucina algerina \n Categoria:Dolci a base di mandorle \n Categoria:Dolci a base di sesamo', 'Il kebab < ref >  < /ref > < ref >  < /ref >  (pron.  < ref >  < /ref > ; in arabo  < span style= " font-size: 120%; " > كباب < /span > , kebāb,  " carne arrostita " ; in turco kebap) è un piatto a base di carne, tipico della cucina turca, divenuto popolare in tutto il mon

In [20]:
# Clean texts from 'Note' until the end and other undeleted tags with regex

clean_text_lst = [re.sub(r"(== Note == | ==Note== )\n *(.)*", "", el, flags=re.DOTALL) for el in text_lst]
clean_text_lst = [re.sub(r"( < ref > | < /ref > )", "", el) for el in clean_text_lst]
clean_text_lst = [re.sub(r"<[^>]+>", "", el) for el in clean_text_lst]
#print(clean_text_lst)

['La samsa è un dolce tipico della cucina algerina e tunisina costituito da alcuni strati di pasta simili alla sfoglia (malsouka) e farcito con mandorle tritate e semi di sesamo  , poi fritto o cotto al forno. Si prepara uno sciroppo di zucchero nel quale si mette la samsa per zuccherarla. La samsa oppure lo sciroppo devono essere ancora caldi per permettere un buon assorbimento   . \n \n ', 'Il kebab  (pron. ; in arabo   كباب  , kebāb,  " carne arrostita " ; in turco kebap) è un piatto a base di carne, tipico della cucina turca, divenuto popolare in tutto il mondo grazie alle immigrazioni dal Medio Oriente. \n \n == Tipi principali == \n thumb|Döner kebap a Istanbul \n Il tipo di kebab più famoso nel mondo è probabilmente il döner kebab. Il fatto che in Turchia venga chiamato anche con un\'espressione che significa  " kebab da passeggio " , per l\'abitudine di mangiarlo anche mentre si cammina per strada, non ha nulla a che vedere con il suo nome, dal momento che döner kebab è traduci